The training loop code for this model was adapted from https://colab.research.google.com/drive/13dZVYEOMhXhkXWfvSMVM1TTtUDrT6Aeh, which sets up a basic scheme for loading data and finetuning a model. All other preprocessing and evaluation code is my own.


# Setup

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.1 MB/s eta 0:00:00


In [ ]:
import os
import time
import datetime
from google.colab import drive

import pandas as pd
import seaborn as sns
import numpy as np
import random

import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/marcoguerini/CONAN/master/Multitarget-CONAN/Multitarget-CONAN.csv')

In [ ]:
import pickle


with open('file.pkl', 'rb') as f:
    rats = pickle.load(f)

In [ ]:
def find_top_k(tokens):
  words = -1 * round(0.3 * (len(tokens['Tokens']) - 2))
  ind = sorted(np.argpartition(tokens['Rationale'], words)[words:])
  c = [tokens['Tokens'][index] for index in ind]
  return ' '.join(c).replace(' ##', "")

  


In [ ]:
find_top_k(rats[90])

'so many being unwilling to procreate'

In [ ]:
rationales = []
for i in range(5003):
  rationales.append(find_top_k(rats[i]))

In [ ]:
print (rats[0])

{'Tokens': ['[CLS]', 'maybe', 'the', 'un', 'could', 'talk', 'to', 'those', 'asian', 'and', 'african', 'nations', 'responsible', 'for', 'percent', '+', 'of', 'the', 'pollution', 'in', 'the', 'oceans', 'instead', 'of', 'insisting', 'on', 'this', 'bullshit', 'about', 'climate', 'change', '.', '[SEP]'], 'Rationale': array([0.01922302, 0.01573417, 0.01436975, 0.03057656, 0.02313791,
       0.01900751, 0.01430929, 0.01816324, 0.06471185, 0.10386582,
       0.05419897, 0.07193087, 0.03740124, 0.02454146, 0.04802616,
       0.06218731, 0.02773118, 0.02986371, 0.03097866, 0.02667735,
       0.02646989, 0.02421796, 0.01695626, 0.01113697, 0.02846369,
       0.0131451 , 0.01670718, 0.05494984, 0.01697002, 0.02656206,
       0.02002127, 0.0039917 , 0.00377202])}


In [ ]:
pd.DataFrame(rationales)

,0
0,asian and african nations responsible percent ...
1,‘ juice ( jews ) are worse that nukes ’
2,foreigners uk benefits leaps percent 4 kouts
3,jews cannot be patriots will
4,"israel , zionism and jewish supremacist control"
...,...
4998,childlike
4999,sub to
5000,essentially childlike unable
5001,sub to


In [ ]:
df

,INDEX,HATE_SPEECH,COUNTER_NARRATIVE,TARGET,VERSION
0,0,Maybe the UN could talk to those asian and afr...,"You're right, ocean pollution is one of the ma...",other,V1
1,1,‘Juice (Jews) are worse that nukes.’ Caption t...,You'd rather be nuked then?,JEWS,V1
2,2,Foreigners on UK benefits leaps 41% in 5 years...,You seem to be worried about government expend...,MIGRANTS,V1
3,3,"Jews cannot be patriots, since their allegianc...",You can have parents and grandparents born els...,JEWS,V1
4,4,"Israel, Zionism and Jewish Supremacist control...",Wouldn't you say its colonialism from many dif...,JEWS,V1
...,...,...,...,...,...
4998,4998,Woman are basically childlike.,They are not childlike and are capable of grea...,WOMEN,V6_mix
4999,4999,women should be submissive to men.,"There are many women who want to be ""submissiv...",WOMEN,V6_mix
5000,5000,"Women are essentially childlike, unable to und...",Maybe you should try talking to a woman instea...,WOMEN,V6_mix
5001,5001,Women should be submissive to men.,Women are human beings deserving respect and e...,WOMEN,V6_mix


In [ ]:
df.dropna(inplace=True) #remove NA values
df["combined"] = df["HATE_SPEECH"] + "<|eohs|>" + df["COUNTER_NARRATIVE"]
data = df["combined"]
data2 = df["HATE_SPEECH"]
data3 = df["COUNTER_NARRATIVE"]

In [ ]:
# Load the GPT tokenizer.
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>') #gpt2-medium


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
batch_size = 2

In [ ]:
# provided class by the code i adapted
class GPT2Dataset(Dataset):

  def __init__(self, txt_list, tokenizer, gpt2_type="gpt2", max_length=768):

    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []

    for txt in txt_list:

      encodings_dict = tokenizer('<|startoftext|>'+ txt + '<|endoftext|>', truncation=True, max_length=max_length, padding="max_length")

      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    
  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx] 

In [ ]:
#data = data[:200]
dataset = GPT2Dataset(data, tokenizer, max_length=768)

# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

from sklearn.model_selection import train_test_split

In [ ]:
train_dataset, val_dataset, t1, v1, t2, v2 = train_test_split(dataset, data2, data3, test_size=0.1)

4,502 training samples
  501 validation samples


In [ ]:
t1

3870    Children should be isolated from gays, they sh...
2960    Muslims are just an inconvenient group to be e...
1778    Women are only useful as sexual slaves and can...
584     Allowing gays to donate blood compromises pati...
1157    When a Muslim says they want to be like us, th...
                              ...                        
3011    Downs syndrome is a curse. People with it are ...
3436    We are surrounded by Muslims infiltrating ever...
4358                Homosexuals are a danger to children.
570     Faggots abuse of their anus and then they want...
2497    Women are capable of nothing and they just nee...
Name: HATE_SPEECH, Length: 4502, dtype: object

In [ ]:
data

0       Maybe the UN could talk to those asian and afr...
1       ‘Juice (Jews) are worse that nukes.’ Caption t...
2       Foreigners on UK benefits leaps 41% in 5 years...
3       Jews cannot be patriots, since their allegianc...
4       Israel, Zionism and Jewish Supremacist control...
                              ...                        
4998    Woman are basically childlike.<|eohs|>They are...
4999    women should be submissive to men.<|eohs|>Ther...
5000    Women are essentially childlike, unable to und...
5001    Women should be submissive to men.<|eohs|>Wome...
5002    Women should not be allowed to vote, this is a...
Name: combined, Length: 5003, dtype: object

In [ ]:
val_dataset[0]

(tensor([50257,  1135,  2314,   307,   705,    83, 13625,   415,     6,   284,
          7045,    11,   484,   389, 10509, 29847,    91,    68,  1219,    82,
            91,    29, 16991,   318,   257,  5737,   286,  4167,   290, 15621,
            13,  1675,   307,  7417,   857,   407,  6646,  6053,   351,   852,
          3765,    11,  1265,   262,   371,  3539,    13, 50256, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
         50258, 50258, 50258, 50258, 50258, 50258, 5

In [ ]:
train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), 
            batch_size = batch_size 
        )

validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), 
            batch_size = batch_size 
        )

In [ ]:
!pip install evaluate
import evaluate
bleu = evaluate.load("bleu")

In [ ]:
predicts = ["hi hello", "hik"]
reals = ["hello hi", "hik"]
bleu.compute(predictions=predicts, references=reals)

{'bleu': 0.0,
 'precisions': [1.0, 0.0, 0.0, 0.0],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0,
 'translation_length': 3,
 'reference_length': 3}

# Finetune GPT2 Language Model; this section adapts code from the demo to create the training loop

In [ ]:
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)

model.resize_token_embeddings(len(tokenizer))

device = torch.device("cuda")
model.cuda()

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
epochs = 5
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8

sample_every = 100

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
# This changes the learning rate as the training loop progresses
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = total_steps)

In [ ]:
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [ ]:
val_dataset[0]

In [ ]:
total_t0 = time.time()

training_stats = []

model = model.to(device)

for epoch_i in range(0, epochs):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()        

        outputs = model(  b_input_ids,
                          labels=b_labels, 
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]  

        batch_loss = loss.item()
        
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

            model.eval()

            
            eohs = "<|eohs|>"
            #prompt = validation_dataloader[random.randint(0, len(validation_dataloader))][0]
            #re = prompt.split(eohs)
            #prompt=re[0]+eohs
            #generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
            #generated = generated.to(device)
            sample_outputs = model.generate(
                                    bos_token_id=random.randint(1,30000),
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length = 200,
                                    top_p=0.95, 
                                    num_return_sequences=1
                                )
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
            
            model.train()

        loss.backward()

        optimizer.step()

        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)       
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0
    scores = []

    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device) #
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)
        
        with torch.no_grad():        

            outputs = model(b_input_ids, 
                             attention_mask = b_masks,
                            labels=b_labels)
          
            loss = outputs[0]  
            
        
        batch_loss = loss.item()
        total_eval_loss += batch_loss  
        #scores.append(bleu.compute(predictions=outputs, references=val_references))
      
    #valid_bleu = bleu.compute(predictions=outputs, references=val_references)
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)    

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    #print("  Validation Scores: {0:.2f}".format(valid_bleu))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            #'valid_bleu': valid_bleu,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Saving & Loading Fine-Tuned Model


In [ ]:

output_dir = './model_save/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)
model_to_save = model 
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


Saving model to ./model_save/


('./model_save/tokenizer_config.json',
 './model_save/special_tokens_map.json',
 './model_save/vocab.json',
 './model_save/merges.txt',
 './model_save/added_tokens.json')

In [ ]:
!ls -l --block-size=K ./model_save/

total 499888K
-rw-r--r-- 1 root root      1K Mar  1 23:40 added_tokens.json
-rw-r--r-- 1 root root      1K Mar  1 23:40 config.json
-rw-r--r-- 1 root root      1K Mar  1 23:40 generation_config.json
-rw-r--r-- 1 root root    446K Mar  1 23:40 merges.txt
-rw-r--r-- 1 root root 498442K Mar  1 23:40 pytorch_model.bin
-rw-r--r-- 1 root root      1K Mar  1 23:40 special_tokens_map.json
-rw-r--r-- 1 root root      1K Mar  1 23:40 tokenizer_config.json
-rw-r--r-- 1 root root    976K Mar  1 23:40 vocab.json


In [ ]:
!ls -l --block-size=M ./model_save/pytorch_model.bin

-rw-r--r-- 1 root root 487M Mar  1 23:40 ./model_save/pytorch_model.bin


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving added_tokens.json to added_tokens.json
Saving config.json to config.json
Saving generation_config.json to generation_config.json
Saving merges.txt to merges.txt
Saving pytorch_model.bin to pytorch_model.bin
Saving special_tokens_map.json to special_tokens_map.json
Saving tokenizer_config.json to tokenizer_config.json
Saving vocab.json to vocab.json


In [ ]:
model = GPT2LMHeadModel.from_pretrained('.')
tokenizer = GPT2Tokenizer.from_pretrained('.')
model.to(device)

# Generate Text

In [ ]:
from tqdm import tqdm

model.eval()

outputs = []
for seq in tqdm(v1):
  prompt =  "<|startoftext|>" + seq + "<|eohs|>"

  generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
  generated = generated.to(device)

  sample_outputs = model.generate(
                                  generated, 
                                  #bos_token_id=random.randint(1,30000),
                                  do_sample=True,   
                                  top_k=50, 
                                  max_length = 300,
                                  top_p=0.95, 
                                  num_return_sequences=1, pad_token_id=tokenizer.eos_token_id
                                  )

  
  for i, sample_output in enumerate(sample_outputs):
    #print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
    outputs.append(tokenizer.decode(sample_output, skip_special_tokens=True).split("<|eohs|>")[1])

100%|██████████| 501/501 [03:24<00:00,  2.44it/s]


In [ ]:
from nltk.translate import bleu_score
print(bleu_score.corpus_bleu(v2.tolist(),
    outputs,
    weights=(1, 0, 0, 0),
    smoothing_function=None,
    auto_reweigh=False,
))

0.15258999372468482


/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

In [ ]:
print(bleu_score.corpus_bleu(v2.tolist(),
    outputs,
    weights=(0, 0, 1, 0),
    smoothing_function=None,
    auto_reweigh=False,
))

2.2250738585072626e-308


/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

In [ ]:
print(bleu_score.corpus_bleu(v2.tolist(),
    outputs,
    weights=(0, 0, 0, 1),
    smoothing_function=None,
    auto_reweigh=False,
))

2.2250738585072626e-308


In [ ]:
bleu.compute(predictions=outputs, references=v2.tolist())

{'bleu': 0.07885302379430917,
 'precisions': [0.25291883527922354,
  0.07450608733688124,
  0.050166464891041165,
  0.04089657884388517],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0137611408199643,
 'translation_length': 14218,
 'reference_length': 14025}

In [ ]:
!pip install rouge_score
rouge = evaluate.load('rouge')
r = rouge.compute(predictions=outputs, references=v2.tolist())
!pip install meteor_score
meteor = evaluate.load('meteor')
m = meteor.compute(predictions=outputs, references=v2.tolist())


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=8f335753b644f56dd99fbc792b9d46bb98b2cc8b53b6f23085799bb5026ee560
  Stored in directory: /root/.cache/pip/wheels/24/55/6f/ebfc4cb176d1c9665da4e306e1705496206d08215c1acd9dde
Successfully built rouge_score


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement meteor_score (from versions: none)
ERROR: No matching distribution found for meteor_score


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
r

{'rouge1': 0.18292790249477747,
 'rouge2': 0.03047267449801986,
 'rougeL': 0.13867328603167436,
 'rougeLsum': 0.13890285062125832}

In [ ]:
m

{'meteor': 0.14923875241835982}

In [ ]:
files.download("test.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!cp test.zip gdrive/MyDrive/models

In [ ]:
!ls gdrive/MyDrive/models

model_save  test.zip


In [ ]:
!zip -r test.zip model_save

  adding: model_save/ (stored 0%)
  adding: model_save/added_tokens.json (deflated 20%)
  adding: model_save/config.json (deflated 51%)
  adding: model_save/generation_config.json (deflated 24%)
  adding: model_save/pytorch_model.bin (deflated 9%)
  adding: model_save/merges.txt (deflated 53%)
  adding: model_save/tokenizer_config.json (deflated 73%)
  adding: model_save/special_tokens_map.json (deflated 78%)
  adding: model_save/vocab.json (deflated 68%)
